# LA's COVID-19 Reopening / Recovery Indicators 

The NY Times wrote about rushed reopenings being [a global trial-and-error experiment](https://www.nytimes.com/2020/05/07/world/europe/coronavirus-reopening-costs.html) and a further analysis showing how US states reopening [fail to meet the WH guidelines](https://www.nytimes.com/interactive/2020/05/07/us/coronavirus-states-reopen-criteria.html). Vox has written about [the importance of COVID testing indicators](https://www.vox.com/2020/5/8/21249880/coronavirus-testing-covid-reopening-economy-lockdowns-social-distancing) as a prerequisite for reopening. 

While we wait for LA's own reopening indicators, we can still see how LA stacks up against NY and Chicago's indicators. [NY State's Reopening Indicators](https://www.nytimes.com/2020/05/04/nyregion/coronavirus-reopen-cuomo-ny.html) were based heavily in part by the White House [Opening Up America Again gating criteria](https://www.whitehouse.gov/wp-content/uploads/2020/04/Guidelines-for-Opening-Up-America-Again.pdf), and Chicago outlined even [tougher benchmarks than the IL statewide strategy](https://www.chicagotribune.com/coronavirus/ct-coronavirus-chicago-reopening-lightfoot-20200508-ztpnouwexrcvfdfcr2yccbc53a-story.html). In terms of outbreak severity, [LA and Chicago come right after NYC](https://www.nytimes.com/2020/05/09/us/coronavirus-chicago.html).

## [White House Gating Criteria](https://www.whitehouse.gov/wp-content/uploads/2020/04/Guidelines-for-Opening-Up-America-Again.pdf)
### Symptoms
* Downward trajectory of influenza-like illnesses (ILI) reported within a 14-day period **and**
* Downward trajectory of COVID-like syndromic cases reported within a 14-day period

### Cases
* Downward trajectory of documented cases within a 14-day period **or**
* Downward trajectory of positive tests as a percent of total tests within a 14-day period (flat or increasing volume of tests) 

### Hospitals
* Treat all patients without crisis care **and**
* Robust testing program in place for at-risk healthcare workers, including emerging antibody testing

| NY | Chicago |
| --- | --- |
| At least 14 days decline in total net hospitalizations on a 3-day rolling average **or** <BR> Less than 15 net new total hospitalizations on a 3-day rolling average </BR> | At least 14 days decline in new cases, on average 
| At least 14 days decline in total net deaths on a 3-day rolling average **or** <BR> Less than 5 new deaths on a 3-day rolling average </BR> | At least 14 days decline in of case rate (per capita?) |
| At least 30% total hospital and ICU beds available  | At least 14 days decline in rate of new cases (per capita?)
| At least 90 days of PPE stockpiled  | Stable or declining rates of hospitalizations  
| Capacity to conduct 30 tests per 1,000 residents per month  | Stable or declining rates of ICU admissions 
| 30 working contact tracers per 100,000 residents | Stable or declining rates of ICU deaths
| Rural regions: Fewer than 2 new COVID patients admitted per 100,000 residents per day | Share of positive results are decreasing below 15% of those tested in the community 
|   | Fewer than 1,800 coronavirus patients in hospital beds 
|   | Fewer than 600 coronavirus patients in ICU beds
|   | Fewer than 450 coronavirus patients on ventilators
|   | Test 135,000 per month, equivalent to 4,500 people per day

In [1]:
import pandas as pd
import utils
import altair as alt

from IPython.display import Markdown

In [2]:
la_county = utils.case_indicators_county("Los Angeles, CA", "3/1/20")
la_city = utils.case_indicators_lacity("4/5/20")
ca = utils.case_indicators_state("California", "3/1/20")
la_msa = utils.case_indicators_msa("Los Angeles", "3/1/20")

alt.HConcatChart(...)

alt.Chart(...)

alt.HConcatChart(...)

alt.HConcatChart(...)

## Testing Indicators

### NY
Indicator: 30 tests per 1,000 residents per month
<br>Assume: population of 8.4M in NYC
<br>Test rate per 1,000 per month: 30 tests per 1,000 residents per month
<br>Tests per day and per month: 8,400 tests/day or 252,000 tests/month.

### Chicago
Indicator: 4,500 tests per day and 135,000 tests per month
<br>Assume: population of 2.7M in Chicago
<br>Test rate per 1,000 per month: 50 tests per 1,000 residents per month
<br>Tests per day and per month: 4,500 tests/day or 135,000 per month

In [3]:
city = "NY"
nyc_pop = 8_400_000
rate_per_1k = 30 / 1_000
print(f"{city} tests per month: {round(nyc_pop * rate_per_1k):,}")
print(f"{city} tests per day: {round(nyc_pop * rate_per_1k / 30):,}")

NY tests per month: 252,000
NY tests per day: 8,400


In [4]:
city = "Chicago"
chicago_pop = 2_700_000
rate_per_1k = 50 / 1_000
print(f"{city} tests per month: {round(chicago_pop * rate_per_1k):,}")
print(f"{city} tests per day: {round(chicago_pop * rate_per_1k / 30):,}")

Chicago tests per month: 135,000
Chicago tests per day: 4,500


In [5]:
la_pop = 10_000_000
city = "LA"

nyc_rate = 30 / 1_000
chicago_rate = 50 / 1_000

# Let's try LA with NYC's rate and Chicago's rate
print(f"{city} tests per month (NYC): {round(la_pop * nyc_rate):,}")
print(f"{city} tests per day (NYC): {round(la_pop * nyc_rate / 30):,}")
print(f"{city} tests per month (Chicago): {round(la_pop * chicago_rate):,}")
print(f"{city} tests per day (Chicago): {round(la_pop * chicago_rate / 30):,}")

LA tests per month (NYC): 300,000
LA tests per day (NYC): 10,000
LA tests per month (Chicago): 500,000
LA tests per day (Chicago): 16,667


In [6]:
# Add share of positive tests -- we can't do because we don't have direct testing data 
# that breaks down total with positive and negative results
# Can do it at weekly level, probably

In [7]:
daily_tests = utils.testing_lacity('4/1/20', 'daily', 10_000, 16_667)
monthly_tests = utils.testing_lacity('4/1/20', 'monthly', 300_000, 500_000)

alt.LayerChart(...)

alt.LayerChart(...)

## Hospital Capacity Indicators

In [8]:
#hospital_df = pd.read_csv(HOSPITAL_URL)
#hospital_df.head()

References:
<br>US_COUNTY_URL = " http://lahub.maps.arcgis.com/sharing/rest/content/items/8aba663239fc428f8bcdc48e213e3172/data "
<br>LA_CITY_URL = " https://lahub.maps.arcgis.com/sharing/rest/content/items/7175fba373f541a7a19df56b6a0617f4/data "
<br>TESTING_URL = " http://lahub.maps.arcgis.com/sharing/rest/content/items/158dab4a07b04ecb8d47fea1746303ac/data "
<br>HOSPITAL_URL = " http://lahub.maps.arcgis.com/sharing/rest/content/items/3da1eb3e13a14743973c96b945bd1117/data "
<br>CROSSWALK_URL = " https://raw.githubusercontent.com/CityOfLosAngeles/aqueduct/master/dags/public-health/covid19/msa_county_pop_crosswalk.csv "

To convert to HTML: 
jupyter nbconvert --to html --no-input --no-prompt my-notebook.ipynb